
<center>
<h1> AWS SageMaker </h1>
    <h2>MLOps using AWS SageMaker </h2>
    <h3>March 23, 2023</h3>
<hr>
<h1>Exploratory Data Analysis of Wind Turbine Dataset</h1>
<hr>
 </center>

![alt](pic/turbines_winji.jpeg)

# Introduction
We start our journey into MLOps with AWS SageMaker by inspecting the dataset with regards to:

1. Data composition, with regards to structure and quality
2. Associations between the different attributes

Let´s get started by installing some of the non-default libraries.

In [ ]:
%%capture
!pip install umap-learn

In [ ]:
%%capture
!pip install seaborn==0.11.2

In [ ]:
import pandas as pd
from pathlib import Path
from sklearn.preprocessing import StandardScaler
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import umap
import os
import warnings
warnings.filterwarnings("ignore")

# Connecting to the s3 bucket


<div class="alert alert-info"> 🎯 <strong> Connect to S3 bucket with wind turbine data </strong>
    
Connect to the bucket where to obfuscated wind turbine data from [winji](https://www.win-ji.com/) is located.
</div>


In [ ]:
# Raw data paths
BUCKET = 's3://sagemaker-d-one-winji-data'
RAW_DATA_FOLDER = 'data'
RAW_DATA_FILE = 'wind_turbines.csv'
RAW_DATA_PATH = os.path.join(BUCKET, RAW_DATA_FOLDER, RAW_DATA_FILE)


df = pd.read_csv(RAW_DATA_PATH)

## 1. Understanding the data composition
we do this by examining:
* quick look into the data 
* dimension, how many rows and columns?
* fraction of missing data
* datatypes of each attribute

### Data sneak peak

In [ ]:
df.head(1)

To have an idea how the data looks like let us display one record of the dataset. The data will become clearer when looking at it in more detail later.

### Data dimensions

In [ ]:
print(df.shape)

We see that we a total of 16 attributes and 52383 rows. Lets next answer how much of the data is missing and what each attribute is composed of.

### Fraction of missing data and datatype per attribute

In [ ]:
(pd.DataFrame([df.isna().mean(), df.dtypes, df.nunique()])
   .T
   .rename({0:'fraction of na',
            1:'datatype',
            2:'n_unique_entries'}, axis=1))

In [ ]:
df['measured_at'].head(1)

With this little breakdown we learn that the data consists of 16 attributes of which 15 are numerical and 1 is a timestamp. There are a total of 52383 entries in this dataset with no missing data across the feature attributes. The columns with missing data are the categories we want to predict. The `subtraction` attribute shows the presence of an error and the `categories_sk` attribute descibes the type of error in more depth. 
Since the aim of this exersice is to showcase this dataset and ultimatly a SageMaker workflow we continue with the `subtraction` attribute and drop the `categories_sk` attribute.

In [ ]:
df.drop("categories_sk", axis=1, inplace=True)

Since this is time series data we wanted to see if the data was homogeniously collected over the time span. Thus we first parsed the `measured_at` column into a machine readable datetime and plotted the number of rows over time.

In [ ]:
df['measured_at'] = pd.to_datetime(df['measured_at'])

plt.hist(df.measured_at.values)
plt.xticks(rotation = 90)
plt.show()

Great! The data is evenly spread over the sampled time horizon from 2020-01 to 2020-07

Now, lets have a look if, on a high level, we can connect the turbine features with an error type.

# 2. Associations between the different data attributes

Ultimatly we wanted to see the highdimensional structure. We did this by:
   1. z-scoring features excluding the time dimension, turbine number and error code
   2. Collapsing the high dimensional data into 2 dimensions using umap
   3. Highlighting excluded attributes on the 2d representation of the data

Feel free to think about what the output of the umap means for our machine learning model. Below the figures you'll find a write up with our interpretation.

In [ ]:
## Preparing the data for the plots
# Split data into labels and features
columns = df.columns.values
y_cols = df.columns.str.contains('wt_sk|subtraction|measured_at')

y = df[list(columns[y_cols])]
X = df[list(columns[~y_cols])]

# 1. Scale features
scaler = StandardScaler()
X_ = scaler.fit_transform(X)

# 2. Get embeddings
embedding = umap.UMAP().fit_transform(X_)

# Curate dataframe for plotting
X_to_plot = pd.DataFrame(embedding, columns=['UMAP1','UMAP2'])
X_to_plot['error_type'] = y['subtraction'].to_list()

# Flag na values with 2 in error type
X_to_plot['error_type'][X_to_plot.error_type.isna()] = 2
X_to_plot['is_error'] = X_to_plot.error_type != 2

X_to_plot = X_to_plot.sort_values(by='is_error', ascending=True)

X_to_plot['wt_sk'] = y['wt_sk'].to_list()

In [ ]:
# Plot 1
p = sns.jointplot(data=X_to_plot,
                x='UMAP1',
                y='UMAP2',
                space= 0,
                alpha=0.5,
                kind='hex'
)
plt.suptitle("Figure 1: Density of feature distribution", y = 1)
plt.show()

Based on the dimensionality reduction technique we deployed above (we used UMAP) we make the following observations:

1. In the first plot we see that the there is not a single point where most of the data is located but that it is spread across the different dimensions.

In [ ]:
# Plot 2
sns.jointplot(data=X_to_plot,
                x='UMAP1',
                y='UMAP2',
                space= 0,
                alpha=0.5,
                hue='wt_sk',
)
plt.suptitle("Figure 2: Features colored by turbine number", y = 1)
plt.show()

2. In the second plot the color indicates the different turbines. Since the colors are equally spread across the dimensions we can assume that there is no strong bias in the data towards one turbine in specific.

In [ ]:
# Plot 3
sns.jointplot(data=X_to_plot,
                x='UMAP1',
                y='UMAP2',
                space= 0,
                alpha=0.5,
                hue='is_error',
)
plt.suptitle("Figure 3: Features colored by error", y = 1)
plt.show()

3. In the third plot the color indicates wether there is an error or not. We find, as expected, that the errors are mostly located on separated islands with some exceptions where the blue and orange points, corresponding to faulty and working turbines, are mixed.

In [ ]:
# Plot 4
sns.jointplot(data=X_to_plot,
                x='UMAP1',
                y='UMAP2',
                space= 0,
                alpha=0.5,
                hue='error_type',
)
plt.suptitle("Figure 4: Features colored by error type", y = 1)
plt.show()

4. Further looking into the error type we find that the different error types are distinct from each other.

We conclude that we can proceed with training a model on the data


<div class="alert alert-info"> 🎉 <strong> Data looks great! Lets try to model it </strong>

</div>
